# Probabilistic Model of Bilateral Lymphatic Spread in Head and Neck

Cancer

Roman Ludwig [](https://orcid.org/0000-0001-9434-328X) (University of Zurich, University Hospital Zurich)  
Yoel Perez Haas (University of Zurich, University Hospital Zurich)  
Jan Unkelbach [](https://orcid.org/0000-0002-4275-990X) (University of Zurich, University Hospital Zurich)

Purpose: According to current guidelines for elective nodal irradiation of oropharyngeal squamous cell carcinoma (OPSCC) patients, large parts of the contralateral lymphatic system are included in the elective clinical target volume (CTV-N), even for lateralized tumors without clinical lymph node involvement in the contralateral neck. In this work, we present a probabilistic model for bilateral lymphatic tumor progression in OPSCC to predict the personal risk for occult disease in any lymph node level (LNL), given the patient’s clinical lymph node involvement, T-stage, and lateralization of the primary tumor.

Methods: We extend a hidden markov model for lymphatic tumor progression, which was previously developed for ipsilateral lymph node involvement, to the contralateral neck. The model represents each of the LNLs I, II, III, IV, V, and VII of both sides of the neck as a hidden binary random variable with states healthy and involved. LNLs are connected to the tumor and among each other via arcs that correspond to spread probabilities. These spread probability rates are learned via Markov chain Monte Carlo (MCMC) sampling from a dataset of 833 OPSCC patients.

Results: The model is able to describe the data on lymph node involvement well with a small number of interpretable parameters. Midline extension of the primary tumor is the main risk factor for contralateral involvement. In addition, the risk of occult metastases in contralateral lymph node level increases with more advanced T-stage and more severe ipsilateral involvement. The probability of involvement in contralateral level III is very low if the upstream level II is clinically negative. Similarly, the probability of involvement in contralateral level IV is very low if the upstream level III is clinically negative.

Conclusions: The model may guide personalized volume reduction of the elective CTV-N. It suggests that the contralateral neck may be spared from elective irradiation for lateralized tumors not crossing the midline. In patients with primary tumors crossing the midline but clinically negative contralateral neck, the contralateral elective irradiation may be limited to level II.

In [2]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
from matplotlib.colors import LinearSegmentedColormap
import upsetplot

from lymph import models
from lyscripts import utils
from lyscripts.plot.utils import COLORS

from scripts import shared, paths, COL, get_lnl_cols

simple_model = shared.get_model("simple", load_samples=True)

In [3]:
# width of figures. May depend on number of text columns
full = 17 # cm
half = full
# half =  8 # cm

# Introduction

When treating head and neck squamous cell carcinomas (HNSCC) with radiotherapy or surgery, not only the primary tumor and the clinically detected lymph node metastases are targeted. Per current guidelines, large volumes of the neck are part of the the elective clinical target volume (CTV-N) ([Vincent Grégoire et al. 2003](#ref-gregoire_ct-based_2003), [2014](#ref-gregoire_delineation_2014); [V. Grégoire and Others 2018](#ref-gregoire_delineation_2018); [Eisbruch et al. 2002](#ref-eisbruch_intensity-modulated_2002); [Biau et al. 2019](#ref-biau_selection_2019); [Chao et al. 2002](#ref-chao_determination_2002); [Vorwerk and Hess 2011](#ref-vorwerk_guidelines_2011); [Ferlito, Silver, and Rinaldo 2009](#ref-ferlito_elective_2009)). This is to minimize the risk of regional recurrences due to untreated microscopic disease which cannot be detected by current in-vivo imaging modalities such as computed tomography (CT), magnetic resonance imaging (MRI), or positron emission tomography (PET). However, minimizing the risk of missing occult disease in the lymph drainage region must be balanced against the toxicity related to unnecessary treatment of healthy tissue.

The aforementioned guidelines for the definition of the CTV-N are based on anatomically defined lymph node levels (LNL) ([Vincent Grégoire et al. 2014](#ref-gregoire_delineation_2014)) and the overall prevalence of lymph node metastases in the LNLs. Based on this, they recommend extensive irradiation of both sides of the neck for a majority of patients. However, this prevalence the guidelines are based on does not take into account an individual patient’s state of tumor progression, which may have a great impact on a patient’s personal risk for occult disease in a given LNL. For example, a patient with substantial nodal involvement in the ipsilateral side and an advanced primary tumor that crosses the mid-sagittal plane would receive extensive elective irradiation of the contralateral LNLs II, III, and IVa. However, if the patient presented with no clinically detectable nodal disease (cN0) and only a small, clearly lateralized T1 tumor, the same contralateral CTV-N would be recommended ([Biau et al. 2019](#ref-biau_selection_2019)).

To better quantify this personalized risk of occult disease we have previously developed an intuitive, probabilistic hidden Markov model ([Ludwig et al. 2021](#ref-ludwig_hidden_2021); [Ludwig, Schubert, Barbatei, Bauwens, Hoffmann, et al. 2023](#ref-ludwig_modelling_2023)) which was originally based on a conceptually similar Bayesian network model ([Pouymayou et al. 2019](#ref-pouymayou_bayesian_2019)). So far, these models were limited to the ipsilateral side of the neck. The main contribution of this work will be to extend the formalism to include risk predictions for the contralateral side as well. We believe this holds the potential to reduce the electively irradiated volumes in the contralateral neck for patients whose individual risk for occult disease is low. In turn, this may substantially reduce the toxicity from irradiation and the associated decrease in quality of life.

The main contributions of this paper are as follows:

1.  In section <a href="#sec-data" class="quarto-xref">section 2</a> we present a multi-centric dataset on lymph node involvement in 833 OPSCC patients. Based on the dataset the main risk factors for contralateral lymph node involvement are identified and the requirements for the bilateral model extension are described (<a href="#sec-requirements" class="quarto-xref">section 2.4</a>).

2.  In <a href="#sec-ext-to-contra" class="quarto-xref">section 4</a> we present a bilateral HMM of lymphatic progression that accounts for the T-stage lateralization of the primary tumor as well as clinical involvement as risk factors for occult contralateral involvement. In <a href="#sec-methods" class="quarto-xref">section 5</a> we describe how the model training and the computational experiments were set up.

3.  In the results in <a href="#sec-results" class="quarto-xref">section 6</a>, we demonstrate that the model is able to describe the patterns of contralateral lymph node involvement observed in the dataset, and we apply the model to estimating risk of occult disease for typical patients. In <a href="#sec-discussion" class="quarto-xref">section 8</a>, implications for volume-deescalated radiotherapy are discussed.

# Data on Lymphatic Progression Patterns

To be able to create models for lymphatic tumor progression that include all relevant LNLs including the contralateral side, we have collected a detailed dataset of 833<!-- num patients --> patients with newly diagnosed oropharyngeal squamous cell carcinomas ([Ludwig et al. 2022](#ref-ludwig_dataset_2022); [Ludwig, Schubert, Barbatei, Bauwens, Werlen, et al. 2023](#ref-ludwig_multi-centric_2023)). It reports the lymph node involvement per LNL of every patient individually in tabular form, in addition to other primary tumor and patient characteristics such as T-category, subside, and lateralization of the primary tumor, and HPV p16 status. Their patient records have been collected at four different institutions and a brief overview over some of their patients’ characteristics are shown in <a href="#tbl-data-overview" class="quarto-xref">table 1</a>. The data from the Inselspital Bern and the Centre Léon Bérard only consist of patients who received a neck dissection. Since this treatment is more commonly chosen for early T-category patients, they also make up a larger portion of the respective dataset. The majority of patients from the University Hospital Zürich and the Vall d’Hebron Barcelona Hospital was treated with definitive radiotherapy. For a subset of 83 patients in the CLB dataset, the lateralization of the primary tumor was not reported.

In [4]:
from typing import Literal

def align(
  column: pd.Series,
  where: Literal["right", "left", "center"],
) -> list[str]:
  """Align a column."""
  return [f"text-align: {where}"] * len(column)

def right_align(column: pd.Series) -> list[str]:
  """Right align column."""
  return align(column, where="right")

def highlight(column: pd.Series, mapping: dict) -> list[str]:
  """Color based on `mapping`."""
  colors = column.map(mapping)
  return colors.map(lambda x: f"color: {x}")

def highlight_bool(
  column: pd.Series,
  c_false: str = COLORS["green"],
  c_true: str = COLORS["red"],
) -> list[str]:
  """Make cell read (`False`) or green (`True`)."""
  return highlight(column, mapping={True: c_true, False: c_false})

def highlight_t_stage(
  column: pd.Series,
  c_early: str = COLORS["green"],
  c_late: str = COLORS["red"],
) -> list[str]:
  """Highlight `early` and `late`."""
  is_early = column == "early"
  return highlight_bool(is_early, c_false=c_late, c_true=c_early)


col_map = {
  COL.inst: "Institution",
  COL.age: "Age",
  COL.nd: "Neck Dissection",
  COL.t_stage: "T-category",
  COL.n_stage: "N-category",
}
short_col_map = {tpl[-1]: val for tpl, val in col_map.items()}

raw = utils.load_patient_data(paths.data)
subdata = raw[col_map.keys()]
subdata.columns = subdata.columns.get_level_values(2)
subdata = (
  subdata
  .rename(columns=short_col_map)
  .reset_index(drop=True)
)

def n0_mean(n_stage: pd.Series) -> float:
  """Compute portion of N0 patients."""
  return (n_stage == 0).mean()

def early_mean(t_stage: pd.Series) -> float:
  """Compute early T-category portion."""
  return t_stage.isin([0,1,2]).mean()

grouped = (
  raw.groupby(
    by=COL.inst,
  ).aggregate(**{
    "Total": pd.NamedAgg(column=COL.age, aggfunc="count"),
    "Age (median)": pd.NamedAgg(column=COL.age, aggfunc="median"),
    "Neck Dissection": pd.NamedAgg(column=COL.nd, aggfunc="mean"),
    "N0": pd.NamedAgg(column=COL.n_stage, aggfunc=n0_mean),
    "Early T-Cat.": pd.NamedAgg(column=COL.t_stage, aggfunc=early_mean),
    "Mid. Ext.": pd.NamedAgg(column=COL.midext, aggfunc="mean"),
  }).convert_dtypes()
)
grouped.index.name = "Institution"
(
  grouped
  .reset_index()
  .style
  .format(
    formatter="{:>.0%}",
    subset=["Neck Dissection", "N0", "Early T-Cat.", "Mid. Ext."],
  )
  .apply(
    func=right_align,
    axis="index",
    subset=grouped.columns[0:],
  )
  .hide()
)

## Consensus on Involvement Status

Pathological involvement is thus available only for a subset of the patients and only for the dissected levels. For the remaining patients, only clinical involvement is available. For the analysis performed in this paper, all available diagnostic information has been combined into a consensus decision representing the most likely state of involvement for each patient and LNL, using literature values for sensitivity and specificity of different modalities ([De Bondt and Others 2007](#ref-de_bondt_detection_2007); [Kyzas and Others 2008](#ref-kyzas_18f-fluorodeoxyglucose_2008)). These values are tabulated in <a href="#tbl-spec-sens" class="quarto-xref">table 2</a>.

This consensus was formed as follows: Suppose the involvement status $X_v$ of LNL $v$ was assessed using different diagnostic modalities $\mathcal{O} = \{ \text{MRI}, \text{CT}, \text{pathology}, \ldots \}$, each characterized by their own pair of sensitivity and specificity values $s_N^{\mathcal{o}}$ and $s_P^{\mathcal{o}}$, with $\mathcal{o} \in \mathcal{O}$. Then we have $|\mathcal{O}|$ observations $z_v^{\mathcal{o}} \in \left[ 0, 1 \right]$, where 0 stands for “healthy” and 1 for “involved”. We can then compute the most likely true involvement $X_v$ using the likelihood function

$$
\begin{aligned}
\ell \left( X_v \mid \{ z_v^{\mathcal{o}} \}_{\mathcal{o} \in \mathcal{O}} \right) = \prod_{\mathcal{o} \in \mathcal{O}}
\left( 1 - X_v \right) \cdot &\left[ z_v^{\mathcal{o}} \cdot \left( 1 - s_P^{\mathcal{o}} \right) + \left( 1 - z_v^{\mathcal{o}} \right) \cdot s_P^{\mathcal{o}} \right] \\
+ X_v \cdot &\left[ z_v^{\mathcal{o}} \cdot s_N^{\mathcal{o}} + \left( 1 - z_v^{\mathcal{o}} \right) \cdot (1 - s_N^{\mathcal{o}}) \right]
\end{aligned}
$$

We now assume the true state $X_v$ to take on the value 1 if $\ell \left( X_v = 1 \mid \ldots \right) > \ell \left( X_v = 0 \mid \ldots \right)$ and 0 otherwise. For example, if we have $z_\text{II}^\text{CT} = 0$ and $z_\text{II}^\text{MRI} = 1$ we would compute the following likelihoods:

$$
\begin{aligned}
\ell \left( X_\text{II} = 1 \mid z_\text{II}^\text{CT} = 0, z_\text{II}^\text{MRI} = 1 \right) &= \left( 1 - s_N^\text{CT} \right) \cdot s_N^\text{MRI} = 15.39\% \\
\ell \left( X_\text{II} = 0 \mid z_\text{II}^\text{CT} = 0, z_\text{II}^\text{MRI} = 1 \right) &= s_P^\text{CT} \cdot \left(1 - s_N^\text{MRI}\right) = 14.44\%
\end{aligned}
$$

In this example, we would thus assume the true state to be involved ($X_\text{II} = 1$).

This method of computing a consensus also ensures that the pathology reports always override any conflicting clinical diagnosis, due to pathology’s high sensitivity and specificity.

| Modality  | Specificity | Sensitivity |
|:----------|------------:|------------:|
| CT        |         76% |         81% |
| PET       |         86% |         79% |
| MRI       |         63% |         81% |
| FNA       |         98% |         80% |
| pathology |        100% |        100% |

Table 2: Specificity and sensitivity values from the literature ([De Bondt and Others 2007](#ref-de_bondt_detection_2007); [Kyzas and Others 2008](#ref-kyzas_18f-fluorodeoxyglucose_2008)).

## Data Availability

The entire data, including additional patients with tumors in other primary locations than the oropharynx, is publicly available: It may be [downloaded from LyProX](https://lyprox.org/patients/dataset) where it can be interactively explored too, [from GitHub](https://github.com/rmnldwg/lydata), [from zenodo](https://zenodo.org/search?q=lydata), or via the *Data-in-Brief* publications Ludwig et al. ([2022](#ref-ludwig_dataset_2022)) and Ludwig, Schubert, Barbatei, Bauwens, Werlen, et al. ([2023](#ref-ludwig_multi-centric_2023)). The Data-in-Brief publications contain a detailed description of the datasets and the data format. However, these publications do not include the most recent dataset addition from Vall d’Hebron Barcelona Hospital.

## Patterns of Contralateral Involvement

These datasets allow us to investigate correlations between the involvement of individual LNLs, or between risk factors and patterns of involvement. In <a href="#fig-data-strat" class="quarto-xref">figure 1</a>, we have plotted the prevalence of each contralateral LNL’s involvement, stratified by T-category, ipsilateral number of involved LNLs, and whether the tumor extended over the mid-sagittal line. A similar but more complete stratification is also tabulated in the appendix in <a href="#tbl-data-strat" class="quarto-xref">table 4</a>.

<figure id="fig-data-strat">

<figcaption>Figure 1: Contralateral involvement stratified by T-category (left panel), the number of metastatic LNLs ipsilaterally (center panel), and whether the primary tumor extended over the mid-sagittal line or was clearly lateralized (right panel).</figcaption>
</figure>

The left panel in <a href="#fig-data-strat" class="quarto-xref">figure 1</a> indicates that T-category is correlated with contralateral involvement (as it is with overal involvement). This is simply because T-category may on average be considered a surrogate for the time between onset of disease and diagnosis. I.e., a patient with a T4 tumor was – on average – diagnosed later than a patient with a T1 tumor. Thus, the former did have more time to develop metastases.

Similarly, ipsilateral involvement correlates with contralateral metastasis. The tumor of a patients with many metastases in ipsilateral LNLs was probably able to spread for longer (or faster) compared to a tumor in a patient with no nodal disease. This, too, may therefore be considered a surrogate for the duration of the disease. In addition, it has been hypothesized that bulky nodal disease ipsilaterally may also redirect lymph fluids to the contralateral side. <!-- citation needed? -->

Lastly, the right panel in <a href="#fig-data-strat" class="quarto-xref">figure 1</a> shows that patients with a tumor crossing the mid-sagittal line show contralateral involvement vastly more often compared to patients with clearly lateralized tumors. This makes intuitive sense, because the lymphatic system in the head and neck region is typically symmetric and thus no major vessels cross the midline. Therefore, interstitial fluids from the primary tumor – which we assume to carry living malignant cells – may only reach the blind-ended lymphatic vessels in the contralateral neck via short-ranged diffusion. Which in turn is only possible when the primary tumor is close enough to the mid-sagittal line or crosses it. <!-- citation(s) needed? or is this intuition good enough? -->

The three risk factors for contralateral involvement, midline extension, T-category, and ipsilateral involvement are correlated. For example, midline extension was present in 48% of advanced T-category tumors but only in 7% of early T-category tumors. The higher prevalence of contralateral metastases in advanced T-category tumors may thus be partially explained by a higher rate of midline extension in these tumors. However, <a href="#fig-data-strat-uncorr" class="quarto-xref">figure 2</a> indicates that advanced T-category and severe ipsilateral involvement do represent additional risk factors . The figure considers only patients with lateralized tumors not extending over the midline. For early T-category and healthy ipsilateral levels I-V, only 1% (1 out of 84 patients) shows involvement of contralateral level II. This increases to 5% (9 out of 165 patients) if ipsilateral level II is involved, to 15% (9 out of 60 patients) if ipsilateral levels II and III are involved, and to 23% (6 out of 26 patients) for advanced T-category with ipsilateral levels II and III involved.<!-- TODO: make values here dynamic. -->

<figure id="fig-data-strat-uncorr">

<figcaption>Figure 2: Contralateral involvement prevalence by LNL for lateralized tumors. Shown are four different scenarios: 1) early T-category with an ipsilateral N0 neck (green), 2) early T-category with ipsilaterally LNL II involved (blue), 3) late T-category with ipsilateral LNL II metastatic (orange), and 4) early T-category with ipsilateral both levels II and III involved (red). This illustrates that even when controlling for the tumor’s lateralization, T-category and ipsilaeral involvement are predictive of contralateral metastases.</figcaption>
</figure>

## Requirements for a Bilateral Model

Based on the observations of the [previous section](#sec-data-strat), any potential model that aims to also predict the risk for contralateral nodal involvement, should be able to take the following into account:

1.  More advanced T-category should lead to higher risk for nodal disease. One approach to achieve this via the expected time of diagnosis has already been developed in the form of a hidden Markov model ([Ludwig et al. 2021](#ref-ludwig_hidden_2021)).
2.  The degree of ipsilateral involvement should give the model information on the time that may have passed between onset and diagnosis of the disease. This should come in addition to what can be inferred about the time from T-category alone.
3.  A tumor that extends over the mid-sagittal line should yield contralateral metastases with much higher probability.

Over the course of this work, we will first briefly recap the mentioned HMM in <a href="#sec-unilateral" class="quarto-xref">section 3</a>, which was so far used to model ipsilateral lymphatic progression only. Then, we intuitively extend it to include the contralateral side as well in <a href="#sec-ext-to-contra" class="quarto-xref">section 4</a>. In this section, we also introduce a way of modelling the tumor’s midline extension as a random variable (<a href="#sec-midline" class="quarto-xref">section 4.2</a>) and lastly talk about how it may affect the contralateral spread in <a href="#sec-params-symmetry" class="quarto-xref">section 4.1</a>.

<!-- TODO: Hint at methods and results... -->

# Unilateral Model for Lymphatic Progression

This paper builds on the previously developed unilateral model for ipsilateral lymph node involvement presented in \[cite 2024 graph extension paper\]. In this section we provide a brief recap of the unilateral model to introduce the notation needed to extend the framework to a bilateral model describing both ipsilateral and contralateral lymph node involvement in <a href="#sec-ext-to-contra" class="quarto-xref">section 4</a>. For further details in the ipsilateral model, the reader is referred to the earlier publications ([Ludwig et al. 2021](#ref-ludwig_hidden_2021), cite 2024 graph extension paper)

<!--
Our first model to predict the lymphatic progression of HNSCC was introduced using Bayesian networks [@pouymayou_bayesian_2019]. We subsequently extended this work to a hidden Markov model (HMM) [@ludwig_hidden_2021] to allow an intuitive inclusion of T-category into the predictions. We will briefly summarize this HMM's formalism before building on it to include the contralateral spread in @sec-ext-to-contra.
-->

We model a patient’s state of involvement at an abstract time-step $t$ as a vector of hidden binary random variables:

<span id="eq-state-def">$$
\mathbf{X}[t] = \begin{pmatrix} X_v[t] \end{pmatrix} \qquad v \in \left\{ 1, 2, \ldots, V \right\}
 \qquad(1)$$</span>

Here, $V$ is the number of LNLs the model considers. The values a LNL’s hidden binary random variables may take on are $X_v[t] = 0$ (`False`), meaning the LNL $v$ is healthy or free of metastatic disease, or $X_v[t] = 1$ (`True`), corresponding to some form of tumor presence (i.e., occult or clinically detected). Since the state vector $\mathbf{X}[t]$ is $V$-dimensional and binary, there are $2^V$ distinct possible lymphatic involvement patterns, which we enumerate from $\boldsymbol{\xi}_0 = \begin{pmatrix} 0 & 0 & \cdots & 0 \end{pmatrix}$ to $\boldsymbol{\xi}_{2^V} = \begin{pmatrix} 1 & 1 & \cdots & 1 \end{pmatrix}$. In addition, each LNL is associated with an observed binary random variable $Z_v$ that describes the clinical involvement of a LNL based on imaging: $Z_v = 0$ (`False`) indicates that the LNL $v$ is healthy based on clinical diagnosis, and $Z_v = 1$ (`True`) indicates that suspicious lymph nodes were detected that are deemed metastatic.

Any hidden Markov model is fully described by three quantities:

1.  A starting state $\mathbf{X}[t=0]$ at time $t=0$ just before the patient’s tumor formed. In our case, this is always the state where all LNLs are still healthy $\boldsymbol{\xi}_0$.
2.  The *transition matrix* <span id="eq-trans-matrix">$$
    \mathbf{A} = \left( A_{ij} \right) = \big( P \left( \mathbf{X}[t+1] = \boldsymbol{\xi}_j \mid \mathbf{X}[t] = \boldsymbol{\xi}_i \right) \big)
     \qquad(2)$$</span> where the value at row $i$ and column $j$ represents the probability to transition from state $\boldsymbol{\xi}_i$ to $\boldsymbol{\xi}_j$ during the time-step from $t$ to $t+1$. Note that we prohibit self-healing, meaning that during a transition, no LNL may change their state from $X_v[t]=1$ to $X_v[t+1]=0$. Consequently, many elements of the transition matrix are zero.
3.  Lastly, the *observation matrix* <span id="eq-obs-matrix">$$
    \mathbf{B} = \left( B_{ij} \right) = \big( P \left( \mathbf{Z} = \boldsymbol{\zeta}_j \mid \mathbf{X}[t_D] = \boldsymbol{\xi}_i \right) \big)
     \qquad(3)$$</span> where in row $i$ and at column $j$ we find the probability to *observe* a lymphatic involvement pattern $\mathbf{Z} = \boldsymbol{\zeta}_j$, given that the true (but hidden) state of involvement at the time of diagnosis $t_D$ is $\mathbf{X}[t_D] = \boldsymbol{\xi}_i$.

The transition matrix $\mathbf{A}$ is parametrized using a directed acyclic graph (DAG) as an abstract representation of the underlying lymphatic network. Directed arcs from the primary tumor to a LNL are associated with a probability $b_v$ for direct spread to LNL $v$ during one time step. Directed arcs from a LNL $v$ to a LNL $r$ are associated with a probability $t_{vr}$ for the tumor to progress from one LNL to the next. In this paper, we build on the DAG shown in <a href="#fig-full-graph" class="quarto-xref">figure 3</a> which was obtained by maximizing the model evidence as described in \[cite 2024 graph extension paper\].

<figure id="fig-full-graph">

<figcaption>Figure 3: Directed acyclic graph (DAG) representing the abstract lymphatic network in the head and neck region. Blue nodes are the LNLs’ hidden random variables, the red node represents the tumor, and the orange square nodes depict the binary observed variables. Red and blue arcs symbolize the probability of lymphatic spread along that edge during one time-step. The orange arcs represent the sensitivity and specificity of the observational modality (e.g. CT, MRI, pathology, …).</figcaption>
</figure>

Using the introduced quantities, we can evolve the distribution of all possible hidden states from $\mathbf{X}[t=0] = \boldsymbol{\xi}_0$ step by step, by successively multiplying this vector with the transition matrix $\mathbf{A}$. For later use, we define at this point the matrices of the dsitributions over all hidden states, given all time-steps $\boldsymbol{\Lambda}$:

<span id="eq-lambda-matrix">$$
\boldsymbol{\Lambda} = P \left( \mathbf{X} \mid \mathbf{t} \right) = \begin{pmatrix}
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^0 \\
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^1 \\
\vdots \\
\boldsymbol{\pi}^\intercal \cdot \mathbf{A}^{t_\text{max}} \\
\end{pmatrix}
 \qquad(4)$$</span>

Where the $k$-th row in this matrix corresponds to the probability distribution over hidden states after $t=k-1$ time-steps.

At the time of the diagnosis $t_D$, we multiply the result with the observation matrix $\mathbf{B}$. We may then look up the likelihood of a patient presenting with the diagnosis $\mathbf{Z}=\boldsymbol{\zeta}_i$ in the $i$-th entry of the final result. However, the remaining issue is that the value of $t_D$ is unknown, i.e. over how many time-steps the HMM should be evolved. We solve this problem by marginalizing over the time of diagnosis. Different distributions over the diagnosis times can then be choosen based on T-category. For instance, the mean of the time-prior to marginalize over the diagnosis time for early T-category patients $P\left( t_D \mid \text{early} \right)$ may be shifted towards earlier times than the one for advanced T-category patients $P\left( t_D \mid \text{early} \right)$. This gives us for example

$$
P\left( \mathbf{X} \mid \text{T}x = \text{early} \right) = \sum_{t=0}^{t_\text{max}} P \left( \mathbf{X} \mid t \right) \cdot P(t \mid \text{early})
$$

In this work, we use binomial distributions $\mathfrak{B} \left( t_D, p_{\text{T}x} \right)$ as time-priors which have one free parameter $p_{\text{T}x}$ for each group of patients we differentiate based on T-category. Also, we fix $t_\text{max} = 10$, which means that the expected number of time-steps from the onset of a patient’s disease to their diagnosis is $\mathbb{E}\left[ t_D \right] = 10 \cdot p_{\text{T}x}$.

## Likelihood Function

With the formalism introduced above, we can write the likelihood function for a patient to present with a diagnosis consisting of an observed state and a T-category $d = \left( \boldsymbol{\zeta}_i, \text{T}x \right)$ as follows:

<span id="eq-single-patient-llh">$$
\ell = P \left( \mathbf{Z} = \boldsymbol{\zeta}_i \mid \text{T}x \right) = \sum_{t=0}^{t_\text{max}} \left[ \boldsymbol{\xi}_0 \cdot \mathbf{A}^t \cdot \mathbf{B} \right]_i \cdot P \left( t \mid \text{T}x \right)
 \qquad(5)$$</span>

Above, the quantity inside $\left[ \ldots \right]_i$ denotes the $i$-th component of the vector that is the result of the vector and matrix multiplications in the square brackets. Note that it is also possible to account for missing involvement information: If a diagnosis (like fine needle aspiration (FNA)) is only available for a subset of all LNLs, we can sum over all those possible complete observed states $\boldsymbol{\zeta}_j$ that match the provided diagnosis.

The single-patient likelihood $\ell$ in <a href="#eq-single-patient-llh" class="quarto-xref">equation 5</a> depends on the spread parameters shown in <a href="#fig-full-graph" class="quarto-xref">figure 3</a> via the transition matrix $\mathbf{A}$ and on the binomial parameters $p_{\text{T}x}$ via time-priors. In this work, we will only differentiate between “early” (T1 & T2) and “advanced” (T3 & T4) T-categories. Therefore, the parameter space of the unilateral model is:

<span id="eq-param-space">$$
\boldsymbol{\theta} = \left( \left\{ b_v \right\}, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.} \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
 \qquad(6)$$</span>

And it is our goal to infer the values of these parameters for a given dataset $\mathcal{D} = \left( d_1, d_2, \ldots, d_N \right)$ of OPSCC patients. The likelihood of these $N$ diagnoses is simply the product of their individual likelihoods as defined in <a href="#eq-single-patient-llh" class="quarto-xref">equation 5</a>. For numerical reasons, we typically compute the data likelihood in log space:

<span id="eq-log-likelihood">$$
\log \mathcal{L} \left( \mathcal{D} \mid \boldsymbol{\theta} \right) = \sum_{i=1}^N \log \ell_i
 \qquad(7)$$</span>

The methodology we use to infer the model’s parameters is detailed in <a href="#sec-sampling" class="quarto-xref">section 5.2</a>.

## Model Prediction in the Bayesian Context

Our stated goal is to compute the risk for a patient’s true nodal involvement state $\mathbf{X}$, *given* their individual diagnosis $d = \left( \boldsymbol{\zeta}_k, \text{T}x \right)$. Using Bayes’ law, this is written as:

<span id="eq-uni-bayes-law">$$
P \big( \mathbf{X} \mid \mathbf{Z}=\boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x \big) = \frac{P \left( \boldsymbol{\zeta}_k \mid \mathbf{X} \right) P \big( \mathbf{X} \mid \boldsymbol{\hat{\theta}}, \text{T}x \big)}{\sum_{i=0}^{2^V} P \left( \boldsymbol{\zeta}_k \mid \mathbf{X}=\boldsymbol{\xi}_i \right) P \big( \mathbf{X}=\boldsymbol{\xi}_i \mid \boldsymbol{\hat{\theta}}, \text{T}x \big)}
 \qquad(8)$$</span>

The term $P \left( \boldsymbol{\zeta}_k \mid \mathbf{X} \right)$ is defined solely by sensitivity and specificity of the diagnostic modality. Terms like this already appeared in the definition of the observation matrx in <a href="#eq-obs-matrix" class="quarto-xref">equation 3</a>. The *prior* $P \big( \mathbf{X} \mid \boldsymbol{\hat{\theta}} \big)$ in the above equation is the crucial term that is supplied by a trained model and its parameters $\boldsymbol{\hat{\theta}}$.

It is possible to compute this *posterior* probability of true involvement not only for one fully defined state $\mathbf{X}$, but also for e.g. individual LNLs: For example, the risk for involvement in level IV would be a marginalization over all states $\boldsymbol{\xi}_i$, where $\xi_{i4}=1$. Formally:

<span id="eq-marg-over-posterior">$$
P \big( \text{IV} \mid \mathbf{Z}=\boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x  \big) = \sum_{k \, : \, \xi_{k4}=1} P \big( \mathbf{X} = \boldsymbol{\xi}_k \mid \boldsymbol{\zeta}_k, \boldsymbol{\hat{\theta}}, \text{T}x  \big)
 \qquad(9)$$</span>

# Extension to a Bilateral Model

A naive approach to model the contralateral lymphatic spread would be to simply employ two independent unilateral models as introduced in <a href="#sec-unilateral" class="quarto-xref">section 3</a>. During training, one could enforce that some parameters are shared between these two models, e.g. the parameterization of the distributions over diagnose times or the spread among the LNLs (\$t\_{vr}). However, this approach lacks a way to describe the correlation between ipsi- and contralateral involvement discussed in section {#sec-data-strat}. Two independent unilateral models would not describe the observation that contralateral involvement becomes more likely with more severe ipsilateral involvement.

<!-- This is displayed in @tbl-data-strat and shows how often the contralateral LNLs I, II, III, and IV were involved, given all possible combinations of midline extension, T-category, and ipsilateral LNL III involvement. Unsurprisingly, the prevalence for contralateral involvement is consistently higher when the tumor extends over the mid-sagittal line or is of later T-category. But it is also more frequent when the ipsilateral side shows more severe involvement, which is here shown via the surrogate LNL III. -->

<!-- Additionally, we could think of an approach to incorporate the primary tumor's mid-sagittal extension as a risk factor. -->

Thus, we extend the formalism in <a href="#sec-unilateral" class="quarto-xref">section 3</a> in such a way that the model’s ipsi- and contralateral side evolve synchronously over time. To achieve that, we start by writing down the posterior distribution of involvement an analogy to <a href="#eq-uni-bayes-law" class="quarto-xref">equation 8</a>, which is now a joint probability of an involvement $\mathbf{X}^\text{i}$ ipsilaterally *and* an involvement $\mathbf{X}^\text{c}$ contralaterally, given a diagnosis of the ipsilateral LNLs $\mathbf{Z}^\text{i}$ and of the contralateral ones $\mathbf{Z}^\text{c}$:

<span id="eq-bilateral-bayes">$$
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right) = \frac{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right)}{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right)}
 \qquad(10)$$</span>

For the sake of brevity, we omit the dependency on the parameters and the T-category here.

The probability of the diagnoses given a hidden state factorises: $P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) = P \left( \mathbf{Z}^\text{i} \mid \mathbf{X}^\text{i} \right) \cdot P \left( \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{c} \right)$, and the two factors are described through observation matrices $\mathbf{B}^\text{i}$ and $\mathbf{B}^\text{c}$.

The term representing the model’s prior probability of hidden involvement does not factorize. However, we assume that there is no direct lymph drainage from an ipsilateral LNL to a contralateral LNL. No major lymph vessels cross the mid-sagittal plane. In the graphical model, this means that we assume no directed arcs between ipsilateral and contralateral LNLs, i.e. tumor spread to the contralateral side is due to the primary tumor alone. We can thus write the joint probability $P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right)$ as a factorising sum:

<span id="eq-bilateral-marginal">$$
\begin{aligned}
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \right) &= \sum_{t=0}^{t_\text{max}} P(t) \cdot P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid t \right) \\
&= \sum_{t=0}^{t_\text{max}} P(t) \cdot P \left( \mathbf{X}^\text{i} \mid t \right) \cdot P \left( \mathbf{X}^\text{c} \mid t \right)
\end{aligned}
 \qquad(11)$$</span>

This assumption makes intuitive sense: The state of the ipsilateral and contralateral sides of the lymphatic network evolves independently over time as there are no major lymph vessels crossing the midline. However, both sides are coupled via time. Qualitatively speaking, according to the model in {#eq-bilateral-marginal}, a joint state with severe contralateral involvement and limited ipsilateral involvement would be unlikely, because severe contralateral involvement becomes likely only at later time steps when limited ipsilateral involvement becomes unlikely.

<!--There may, however, be diffusion of lymph fluid accross this line or bulky involvement that redirects lymphatic drainage significantly.-->

Using {#eq-bilateral-marginal} along with <a href="#eq-lambda-matrix" class="quarto-xref">equation 4</a>, we can write the above distribution algebraically as a product:

<span id="eq-bilateral-marginal-algebra">$$
P \left( \mathbf{X}^\text{i} = \boldsymbol{\xi}_n, \mathbf{X}^\text{c} = \boldsymbol{\xi}_m \right) = \left[ \boldsymbol{\Lambda}^\intercal_\text{i} \cdot \operatorname{diag} P(\mathbf{t}) \cdot \boldsymbol{\Lambda}_\text{c} \right]_{n,m}
 \qquad(12)$$</span>

## Parameter Symmetries

In general, the matrices $\boldsymbol{\Lambda}_\text{i}$ and $\boldsymbol{\Lambda}_\text{c}$ could be parameterized using a disjoint set of parameters. I.e., the ipsi- and contralateral spread rates are entirely different. However, using three sensible assumptions, we can reduce the parameter space by sharing some parameters between the sides:

1.  We assume that both ipsilateral and contralateral spread is described through the same graph shown in {#fig-full-graph}.
2.  We assume the spread *among* the LNLs to be same on both sides. It is reasonable to assume the lymphatic system is symmetric. Thus, the spread rates from one LNL to the other should be symmetric, too. Formally, this means  
    <span id="eq-symmetries">$$
    \begin{aligned}
    b_v^\text{c} &\neq b_v^\text{i} \\
    t_{rv}^\text{c} &= t_{rv}^\text{i}
    \end{aligned}
     \qquad(13)$$</span> for all $v \leq V$ and $r \in \operatorname{pa}(v)$.
3.  The tumor’s spread to the contralateral side in case of an extension over the midline is larger than if it was clearly lateralized, but smaller than its spread to the ipsilateral side. This assumption stems from a simple thought experiment: Consider moving the tumor from a clearly lateralized position accross the mid-sagittal plane to the same position, but on the contralateral side. In the beginning we would have $b_v^\text{c} < b_v^\text{i}$, while in the end, the situation is reversed. If a tumor extends over the mid-sagittal line, its contralateral spread rate can be expected to be in between these two extremes. We encode this in a *mixing parameter* $\alpha \in [0,1]$ that captures a “degree of asymmetry”:  
    <span id="eq-mixing">$$
    b_v^{\text{c},\epsilon=\texttt{True}} = \alpha \cdot b_v^\text{i} + (1 - \alpha) \cdot b_v^{\text{c},\epsilon=\texttt{False}}
     \qquad(14)$$</span> This means the model now uses three different sets of parameters to describe the spread from the tumor to the LNLs: $b^\text{i}_v$ for the spread to the ipsilateral LNLs, $b_v^{\text{c},\epsilon=\texttt{False}}$ for the spread to the contralateral LNLs as long as the tumor is clearly lateralized, and finally $b_v^{\text{c},\epsilon=\texttt{True}}$ when it crosses the midline. Note, however, that these three sets of spread rates only account for $2 \cdot 2^V + 1$ parameters, since they are coupled via the mixing parameter $\alpha$.

Our parameter space has now expanded to

<span id="eq-bi-param-space">$$
\boldsymbol{\theta} = \left( \left\{ b_v^\text{i} \right\}, \left\{ b_v^\text{c} \right\}, \alpha, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.} \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
 \qquad(15)$$</span>

which is less than double in size, compared to the unilateral model. From these parameters, we now build three different transition matrices: The unchanged matrix $\mathbf{A}_\text{i}$ for the ipsilateral side, one $\mathbf{A}_\text{c}^{\epsilon=\texttt{False}}$ for the contralateral side that covers the progression as long as the tumor is lateralized, and $\mathbf{A}_\text{c}^{\epsilon=\texttt{True}}$ in case of a tumor that has crossed the mid-sagittal line.

## Modelling Midline Extension

It can be assumed that most tumors that cross the midline at the time of diagnose have started as lateralized tumors and grew over the midline at a later time step. Hence, the transition matrix $\mathbf{A}_\text{c}^{\epsilon=\texttt{True}}$ only applies for a subset of the time steps. Therefore, we also model the tumor’s extension over the mid-sagittal line as a binary random variable. A tumor starts lateralized and at every time-step there is a finite probability $p_\epsilon$ that the tumor grows over the midsaggital plane. The overall probabilities to find a patient with a clearly lateralized tumor or one that extends over the mid-sagittal line after $t$ time-steps are then respectively given by

$$
\begin{aligned}
P(\epsilon = \texttt{False} \mid t) &= (1 - p_\epsilon)^t \\
P(\epsilon = \texttt{True} \mid t) &= 1 - P(\epsilon = \texttt{False} \mid t)
\end{aligned}
$$

Using this, it is straightforward to write down the matrix of state distributions for all time-steps, as in <a href="#eq-lambda-matrix" class="quarto-xref">equation 4</a> covering the contralateral hidden state evolution:

$$
\boldsymbol{\Lambda}_\text{c}^{\epsilon=\texttt{False}} =
\left(
\begin{array}{r}
\boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{0\phantom{t_\text{max}}} \\
(1-p_\epsilon) \cdot \boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{1\phantom{t_\text{max}}} \\
\hfill \vdots \hfill \\
(1-p_\epsilon)^{t_\text{max}} \cdot \boldsymbol{\pi}^\intercal \cdot \left( \mathbf{A}_\text{c}^{\epsilon=\texttt{False}} \right)^{t_\text{max}\phantom{0}} \\
\end{array}
\right)
$$

where we used the transition matrix $\mathbf{A}_\text{c}^{\epsilon=\texttt{False}}$ that depends on the base spread parameters $b_v^{\text{c},\epsilon=\texttt{False}}$.

The case when midline extension is eventually present is more complicated: We already marginalize over the exact time-step when the tumor grows over the mid-sagittal line. But whenever that happens, we also need to change the contralateral transition matrix to use the increased spread rates $b_v^{\text{c}, \epsilon=\texttt{True}}$ from the tumor to the contralateral LNLs, given by the linear mixing in <a href="#eq-mixing" class="quarto-xref">equation 14</a>. We can achieve the correct marginalization by iteratively building the joint distribution $P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid t \right)$. We start from $t=0$, where we know that all LNLs are healthy (the contralateral neck is in the state $\mathbf{X}_\text{c}=\boldsymbol{\xi}_0$) and the tumor is lateralized ($\epsilon=\texttt{False}$):

$$
P \left( \mathbf{X}^\text{c} = \boldsymbol{\xi}_0, \epsilon=\texttt{False} \mid t=0 \right) = 1
$$

whereas all other states have zero probability. Then we consider an arbitrary later time-step $t=\tau+1$. There are two possible scenarios we need to marginalize over:

1.  The tumor was still lateralized at $t=\tau$ and just grew over the midline. Under this scenario, the probability for having a midline extension at $t=\tau+1$ is given by $p_\epsilon$. We use this to weight the contralateral state distribution that was so far evolved without increased contralateral spread.
2.  The mid-sagittal line was already crossed by the tumor. In that case, the probability is 1 to remain in that lateralization state. Thus, for this case we simply add the same distribution we are computing, but from one time-step earlier, arriving at a recursion relation:

$$
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid \tau + 1 \right) = \big[ p_\epsilon P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{False} \mid \tau \right) + P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid \tau \right) \big]^\top \cdot \mathbf{A}_\text{c}^{\epsilon=\texttt{True}}
$$

We can collect the iteratively computed distributions for the midline extension case to define the matrix over the states given all time-steps, in analogy to <a href="#eq-lambda-matrix" class="quarto-xref">equation 4</a>:

$$
\boldsymbol{\Lambda}_\text{c}^{\epsilon=\texttt{True}} = \begin{pmatrix}
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid 0 \right) \\
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid 1 \right) \\
\vdots \\
P \left( \mathbf{X}^\text{c}, \epsilon=\texttt{True} \mid t_\text{max} \right) \\
\end{pmatrix}
$$

Using this, we can again write the joint of ipsi- and contralateral involvement - now also for the case of mid-sagittal extension - algebraically as before in <a href="#eq-bilateral-marginal-algebra" class="quarto-xref">equation 12</a>:

$$
P \left( \mathbf{X}^\text{i} = \boldsymbol{\xi}_n, \mathbf{X}^\text{c} = \boldsymbol{\xi}_m, \epsilon \right) = \left[ \boldsymbol{\Lambda}^\intercal_\text{i} \cdot \operatorname{diag} P(\mathbf{t}) \cdot \boldsymbol{\Lambda}_\text{c}^\epsilon \right]_{n,m}
$$

We can use this term to compute the likelihood of all patients with and without midline extension separately. And if for some patients the information of tumor lateralization is not available, we can simply sum the above term once for $\epsilon=\texttt{False}$ and once for $\epsilon=\texttt{True}$ to marginalize over the unknown variable.

The final parameter space of our extended model has now reached this size:

<span id="eq-ext-param-space">$$
\boldsymbol{\theta} = \left( \left\{ b_v^\text{i} \right\}, \left\{ b_v^\text{c} \right\}, \alpha, \left\{ t_{vr} \right\}, p_\text{early}, p_\text{adv.}, p_\epsilon \right) \quad \text{with} \quad \genfrac{}{}{0pt}{2}{v\leq V}{r\in\operatorname{pa}(v)}
 \qquad(16)$$</span>

# Computational methods

<!-- Maybe I should break this apart into separate chapters on data, sampling, etc. -->

In this section, we detail how the experiments were performed. Every figure, table, and result is fully reproducible via the GitHub repository [`rmnldwg/bilateral-paper`](https://github.com/rmnldwg/bilateral-paper). It also contains the raw manuscript and instructions on how to recreate all figures, tables, and the final document.

## Involvement Data Consensus

It is possible to provide our model with multiple different diagnostic modalities, each being characterized by different pairs of sensitivity and specificity. However, we instead chose to combine them into a single “consensus” diagnosis before parameter inference. We opted for this because the literature values of sensitivity and specificity ([De Bondt and Others 2007](#ref-de_bondt_detection_2007); [Kyzas and Others 2008](#ref-kyzas_18f-fluorodeoxyglucose_2008)) of imaging modalities like MRI and CT do not plausibly match some of our observations: In the USZ cohort, 78% of OPSCC patients where diagnosed with ipsilateral LNL II involvement via diagnostic imaging. This is virtually impossible with sensitivities around 80% and specificities lower than 100%.

Our pre-training consensus was formed by considering all reported diagnostic information for a particular patient and LNL. When conflicts arose, we computed the *most likely* true state of involvement using the literature sensitivity and specificity values ([De Bondt and Others 2007](#ref-de_bondt_detection_2007); [Kyzas and Others 2008](#ref-kyzas_18f-fluorodeoxyglucose_2008)).

## MCMC Sampling

For parameter inference, we used the Python package [`emcee`](https://emcee.readthedocs.io/en/stable/) ([Foreman-Mackey et al. 2013](#ref-foreman-mackey_emcee_2013)). It implements efficient MCMC sampling algorithms that employ multiple parallel samplers for affine invariance and better performance on multi-core CPUs. The sample proposal algirothms used by us are based on differential evolution moves ([ter Braak and Vrugt 2008](#ref-ter_braak_differential_2008); [Nelson, Ford, and Payne 2013](#ref-nelson_run_2013)). The [`emcee`](https://emcee.readthedocs.io/en/stable/) library was provided with the likelihood implemented by our [`lymph-model`](https://lymph-model.readthedocs.io/en/stable/) Python package.

For each dimension in the parameter space of the model, we initialized 12 of these parallel samplers, called “walkers”, with random values in the unit cube. Every time all of these walkers advanced 50 steps, the autocorrelation time of the chains was estimated. For short chains, this estimate is not trustworthy, but stabilizes for longer chains. We therefore considered a sampling to be converged when two criteria were met:

1.  The change in the autocorrelation time was less then 5.0e-2.
2.  The estimate of the autocorrelation dropped below $n$ / 50 where $n$ is the length of the chain up to that point.

All samples up to this convergence - called the *burn-in phase* - were discarded. We only kept another 10 samples after that, which were spaced 10 steps apart.

First, in <a href="#fig-model-burnin-history" class="quarto-xref">figure 4</a>, we verify the sampling converged successfully by inspecting two monitoring quantities: The autocorrelation time of the MCMC chain and the acceptance fractions of the parallel walkers.

<figure id="fig-model-burnin-history">

<figcaption>Figure 4: Monitoring quantities during the burn-in phase of the parameter sampling. Left: The autocorrelation time of the sampling chain estimated at different sampling steps. We consider the chain converged when the estimate of the autocorrelation time is stable and drops below the trust threshold of <span class="math inline">\(n/50\)</span> where <span class="math inline">\(n\)</span> is the number of steps. Right: Fraction of accepted MCMC proposals averaged over all parallel walkers. Values around 30% indicate good mixing of the walkers.</figcaption>
</figure>

## Computing the Observed and Predicted Prevalence of Involvement Patterns

We want to assess the model’s capability to approximate the distribution of lymphatic involvement patterns seen in the data. To that end, we compare the prevalence of some invovlement patterns under selected scenarios with the model’s prediction for how often these involvements it expects to see, given these scenarios.

In this context, a “scenario” includes the patient’s T-category $\text{T}x$ and whether the patient’s tumor extended over the mid-sagittal line, i.e. $\epsilon=\texttt{True}$ or $\epsilon=\texttt{False}$.

An involvement pattern specifies for each ipsi- and contralateral LNL whether it is “healthy”, “involved”, or “masked”. If it is “masked”, we essentially state that we are not interested in the involvement of that LNL and the prevalence will be marginalized over this LNL’s involvement.

For example, we may be interested in the prevalence of contralateral LNL II involvement (i.e., contra LNL II “involved” and all other LNLs “masked”) under the scenario of early T-category (T0-T2) and no midline extension ($\epsilon=\texttt{False}$). To compute this prevalence in the data, we select all patients of this scenario (in our data, this amounts to 379<!-- TODO: dyamically compute this --> patients). Of those, 28<!-- TODO: dynamically compute this --> were found to harbor metastases in their contralateral LNL II. Therefore, the prevalence is 7%<!-- TODO: dynamically compute this -->.

When displaying this data prevalence, we often choose to draw a *beta posterior* over the “true” prevalence, hinting at the fact that our data merely represents a limited sample. The beta posterior follows from a uniform beta distribution as prior and a binomial likelihood for the number of patients with the involvement of interest, given the parameer for the “true” prevalence. The resulting distribution has its maximum at the observed prevalence, but in addition gives a visual intuition for the variance of of the observed quantity. I.e., when we observe 3 out of 10 events, the beta posterior is much wider than if we observe 300 out of 1000 for he same prevalence. It also allows us to check not only if the model is accurate, but also whether it reflects the uncertainty contained in the data.

Predicting the prevalence using our model amounts to computing the following probability:

$$
P \left( \text{II}^\text{c} \mid \epsilon=\texttt{False}, \text{T}x=\text{early} \right) = \frac{P \left( \text{II}^\text{c}, \epsilon=\texttt{False} \mid \text{T}x=\text{early} \right)}{P \left( \epsilon=\texttt{False} \mid \text{T}x=\text{early} \right)}
$$

In the enumator, we marginalize over all ipsi- and contralateral LNLs’ involvements, except for LNL II contralaterally. This is similar to the marginalization in <a href="#eq-marg-over-posterior" class="quarto-xref">equation 9</a>, although we are summing over different quantities. In the denominator, we can simply insert the joint distribution over midline extension and diagnose time $P \left( \epsilon, t \right)$ marginalized over $t$ using the early T-category’s time-prior.

Since we compare it to the data, which does not report true but only observed involvement – although pathologically investigated LNLs may be as close as possible to the ground truth – we do not consider posteriors of the form $P \left( \mathbf{X} \mid \mathbf{Z} \right)$ here. Instead, we compute probabilities of observed involvement $P \left( \mathbf{Z} \right)$, as in the likelihood <a href="#eq-single-patient-llh" class="quarto-xref">equation 5</a>.

When plotted, we usually display histograms over the model’s predictions. Each of their values was computed from a different parameter set drawn during MCMC sampling, effectively giving us a distribution over the prevalences. Ideally, the histograms approximate the location and width of the Beta posteriors when attempting to describe the data they were trained on.

Note that we decided to omit the y-axis ticks and labels in these figures over prevalences and risks. The y-axis in these plots measures the probability density and its numerical values are not intuitively interpretable. Instead, we occasionally use the freed space to label e.g. rows of subplots.

# Results: Model evaluation

In <a href="#tbl-midline-params" class="quarto-xref">table 3</a>, we tabulate the mean and standard deviation of the sampled parameters for the full midline model. Considering the parameters of the bilateral model that describe the ipsilateral spread ($b^i_v$, $t_{vr}$ and $p_\text{adv.}$), we note that the bilateral model mostly reproduces the parameter values reported in the earlier publication ([Ludwig, Schubert, Barbatei, Bauwens, Hoffmann, et al. 2023](#ref-ludwig_modelling_2023)) for the unilateral model. The small differences may be mostly due to the differences in training dataset. Therefore, the analysis of the models capability to adequately describe ipsilateral lymph node involvement is not repeated here and we focus instead on the analysis of contralateral involvement. We can further note that the contralateral spread parameters $b^c_v$ are small compared to the ipsilateral parameters $b^i_v$, reflecting the low prevalence of contralateral lymph node involvement for lateralized tumors.

In [5]:
def map_param_names(name: str) -> str:
  """Make parameter names more readable."""
  try:
    return {
      "midext_prob": "Mid. ext. probability",
      "mixing": "Mixing ⍺",
      "late_p": "late T-cat. binom. prob.",
    }[name]
  except KeyError:
    return re.sub(
      r"(ipsi|contra)_", r"\g<1>: ", name
    ).replace(
      "to", " ➜ "
    ).replace(
      "_spread", ""
    )

model = shared.get_model(which="full", load_samples=True)
samples = shared.get_samples(which="full")

names = [map_param_names(p) for p in model.get_params()]
means, stds = samples.mean(axis=0), samples.std(axis=0)

early_midext_prob = model.state_dist(t_stage="early")[1].sum()
late_midext_prob = model.state_dist(t_stage="late")[1].sum()

params_table = pd.DataFrame({"Parameter": names, "Mean": means, "Std. Dev.": stds})
(
  params_table.style
  .format("{:.2%}", subset=["Mean"])
  .format("± {:.2%}", subset=["Std. Dev."])
  .apply(right_align, subset=["Mean", "Std. Dev."])
  .hide()
)

## Illustration of the model

In this subsection we illustrate parts of the mathematical framework defined in the previous section. In <a href="#fig-model-midext-evo" class="quarto-xref">figure 5</a> (top panel), we visualize the prior distribution over diagnose times $P(t)$. By parameter choice, early T-category tumors are diagnosed on average after 3 time steps. Advanced T-category tumors are diagnosed on average after 4.5 time steps ($p_\text{adv.}=0.45$) <!-- TODO: make dynamic -->, a parameter learned from the data to describe the higher involvement for advanced T-category tumors. The probability per time step for the tumor to grow over the midline, $p_\epsilon = 0.0816$ <!-- TODO: make dynamic --> yields the probability of midline extension $P(\epsilon \mid t)$ at a given time step $t$ (red line). The parameter is adjusted such that the model yields the correct proportion of patients with tumors extending over the midline seen in the dataset. The bottom panel in <a href="#fig-model-midext-evo" class="quarto-xref">figure 5</a> shows the joint probability $P(\epsilon, t)$ to be diagnosed at time step $t$ with a given state of midline extension and T-category.

<figure id="fig-model-midext-evo">

<figcaption>Figure 5: The top panel shows the prior probability to get diagnosed at time-step <span class="math inline">\(t\)</span> for early and late T-category tumors as bars. Also in the top panel, we plot the conditional probability of the tumor’s midline extension (<span class="math inline">\(\epsilon=\texttt{True}\)</span>), given the time-step <span class="math inline">\(t\)</span> as a line plot. In the bottom panel, we show the joint probability of getting diagnosed in time-step <span class="math inline">\(t\)</span> <em>and</em> having a tumor that crosses the midline.</figcaption>
</figure>

The model introduced in the previous section defines a model of the joint probability distribution over midline extension and ipsi- and contralateral lymph node involvement, $P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \right)$. How this quantity is computed is illustrated in <a href="#fig-model-state-dist" class="quarto-xref">figure 6</a>, visually representing <a href="#eq-bilateral-marginal-algebra" class="quarto-xref">equation 12</a>. To keep it manageable to interpret, this example only considers the LNLs II, III, and IV ipsi- and contralaterally, resulting in $2^3 = 16$ distinct states per side and thus $2 \times 8 \times 8 = 128$ total states. LNLs I, V, and VII and the associated spread parameters have been removed from the model while the remaining parameters are set to the mean values in table xxx. <a href="#fig-model-state-dist" class="quarto-xref">figure 6</a> shows the joint distribution as two separate heatmaps for the two midline extension states for early/advanced??? T-category tumors. The most likely state is a lateralized tumor with involvement of ipsilateral level II and no contralateral involvement, which has a probability of approximately 25%. The second most probable state is a lateralized tumor with ipsilateral involvement of levels II and III and no contralateral involvement. The most likely state with contralateral involvement are tumors with midline extension with involvement of contralateral level II and ipsilateral level II and III.

<figure id="fig-model-state-dist">

<figcaption>Figure 6: Visual representation of <a href="#eq-bilateral-marginal-algebra" class="quarto-xref">equation 12</a> for the case of midline extention. The left and right matrix represent the evolution of the possible hidden states for the ipsi- and contralateral neck respectively. In particular, the right matrix describes the evolution of the contralateral hidden states <em>and</em> midline extension. In the center, the time-prior is plotted as a diagonal matrix. The result of this matrix computation is the joint distribution <span class="math inline">\(P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon=\texttt{True} \right)\)</span></figcaption>
</figure>

## Prevalence predictions for contralateral involvement

The bilateral model was designed to fulfil the requirements laid out in <a href="#sec-requirements" class="quarto-xref">section 2.4</a>. Now, we investigate to what extent the model can quantitatively describe the patterns of lymph node involvement observed in the dataset. To that end, we compare the model’s predictions for contralateral involvement against observations in the data. This is done given scenarios that differ in T-category and/or midline extension and/or ipsilateral involvement.

### Dependence of Contralateral Involvement on T-Category and Midline Extension

In <a href="#fig-model-prevalences-overall" class="quarto-xref">figure 7</a>, we plot the prevalence of contralateral involvement of the LNLs II, III, and IV for the four scenarios made up of the possible combinations of early and late T-category, as well as lateralized and midline extending tumors.

<figure id="fig-model-prevalences-overall">

<figcaption>Figure 7: Comparison of predicted (histograms) vs observed (beta posteriors) prevalences. Shown for the contralateral LNLs II (blue), III (orange), and IV (green). The top row shows scenarios with early T-category tumors, the bottom row for late T-category ones. The left column depicts scenarios where the primary tumor is clearly lateralized, the right column scenarios of tumors extending over the mid-sagittal line. This figure illustrates the model’s ability to describe the prevalence of involvement for different combinations of the risk factors T-category and midline extension.</figcaption>
</figure>

<a href="#fig-model-prevalences-overall" class="quarto-xref">Figure 7</a> shows nicely that the model is capable of accurately accounting for the most important risk factors, i.e. T-category and midline extension. As observed in the data, the model predicts that the prevalence of contralateral LNL II involvement increases from below 8% <!--  TODO: make this dynamic --> for early T-category lateralized tumors to almost 40% <!-- TODO: make this dynamic --> when the tumor is of advanced T-category and crosses the mid-sagittal line. Similarly, the prevalence of contralateral LNL III involvement increases from around 2% <!--  TODO: make this dynamic --> for early T-category lateralized tumors to almost 15% <!-- TODO: make this dynamic --> when the tumor is of advanced T-category and crosses the mid-sagittal line.

<!--  However, for early T-category scenarios with midline extension, the model does seem to overestimate contralateral LNL II and III invovlement. This likely stems from the the small sample size of this relatively rare scenario as hinted at by the wide beta posteriors. -->

### Correlation between Ipsi- and Contralateral Involvement

<figure id="fig-model-prevalences-with-ipsi">

<figcaption>Figure 8: Comparison of the computed and observed prevalences for scenarios that illustrate the model’s capability of accounting for the correlation between ipsi- and contralateral involvement. We show two scenarios: One where the ipsilateral neck shows no involvement (at least LNLs I to V are healthy, LNL VII was ignored because data on it is missing for some patients) in blue and one where at least LNL III was involved in orange. These two scenarios are plotted for all combinations of T-category (early in top row, advanced in bottom row) and tumor lateralization (lateralized in left column, extending over mid-sagittal line in right column).</figcaption>
</figure>

In <a href="#fig-model-prevalences-with-ipsi" class="quarto-xref">figure 8</a> we display the model’s ability to capture the correlation between ipsi- and contralateral involvement. It shows that the prevalence of metastases in the two sides of the neck is correlated via the time of diagnosis, despite the model not having any direct connections between the two side. However, there are some discrepancies in the model’s prediction: It cannot quite capture the correlation of ipsi- and contralateral involvement that is seen in the data. This is notable for example in the bottom right subplot where the prevalence is overestimated for the scenario of an (almost) healthy ipsilateral neck and underestimated when at least LNL III shows metastases. One reason for this may be the width of the time-prior: If it was wider, the posterior over the diagnosis time given the ipsilateral involvement could more flexibly shift to reflect a more severe contralateral involvement.

<!-- NOTE: This last sentence is purely speculative! We need to discuss this. -->

### Prevalence of Midline Extension

<figure id="fig-model-prevalences-midext">

<figcaption>Figure 9: Comparing the predicted (histograms) and observed (lines depicting beta posteriors) prevalence of midline extension for early (blue) and late (orange) T-category. While the prevalence is predicted correctly when marginalizing over T-category, the model cannot capture the degree of separation observed in the data. Since the tumor’s midline extension is virtually always part of the diagnosis and hence <em>given</em> when predicting a patient’s risk, we do not consider this discrepancy a major issue.</figcaption>
</figure>

Lastly, in <a href="#fig-model-prevalences-midext" class="quarto-xref">figure 9</a>, we plot the prevalence of midline extension in the data versus our model’s prediction. It is obvious the model cannot match the large spread between early and advanced T-category seen in the data. This is because to achieve that, it would need to increase the advanced T-category patient’s prior distribution over diagnosis times and at the same time reduce the probability of the tumor to cross the midline during a time-step. But since the time-priors parameter is also coupled with the spread probabilities among the LNLs, the model does not have that freedom.

<!-- Should the paragraphs below be part of the discussion? -->

However, we do not consider this discrepancy a major limitation of the model: We will not realistically be interested in the probability of midline extension, as it is always possible to assess it with high certainty. That is also the reason why we initially modelled the midline extension *not* as a random variable, but as a global risk factor that would have been turned on or off from the onset of a patient’s disease evolution. This, however, lead to overly high risks for contralateral involvement in advanced T-category patients with midline extension, because then the model assumes an increased spread to the contralateral side from the onset of the disease. Which is probably not true in a majority of those cases. Thus, treating it as a random variable that only becomes true during a patient’s disease evolution resulted in a better description of the data.

Formally, the wrong prediction of midline extension prevalence makes little difference, since it is always given: Instead of $P\left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \right)$, we typically compute $P\left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right)$, which does not suffer from the wrong probability of midline extension, as the distribution over hidden states is renormalized:

$$
P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c} \mid \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right) = \frac{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c} \mid \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \right) P \left( \mathbf{X}^\text{i}, \mathbf{X}^\text{c}, \epsilon \right)}{P \left( \mathbf{Z}^\text{i}, \mathbf{Z}^\text{c}, \epsilon \right)}
$$

Note that a distribution over $\epsilon$ appears both in the enumerator and the denominator, which largely cancel each other, leaving only the midline extension’s effect on the distribution over hidden states in the prediction.

Also, the discrepancy in midline extension prevalence between early and advanced T-category is particularly pronounced in oropharyngeal SCC patients. For example, in oral cavity SCC, the midline extension only increases from 15.4% (20 out of 130) to 33.3% (13 out of 39).

# Results: Prediction of Risk for Occult Disease

For a clinical application of the model we are interested in the risk of occult metastases rather than the observed patterns of lymph node involvement. We want to estimate the risk for occult disease in clinically negative LNLs, given the patient’s individual diagnosis. In terms of our model, the diagnosis consists of the T-category, the lateralization of the tumor (if it extend over the mid-sagittal plane), and which LNLs are clinically involved based on imaging and possibly fine needle aspiration (FNA). For the results shown in this section, we assume that involvement of a LNL is clinically diagnosed through imaging with a sensitivity of 81% and a specificity of 76% ([De Bondt and Others 2007](#ref-de_bondt_detection_2007)). For FNA we assume a specificity of 98% and a sensitivity of 80%. This amounts to the assumption that lymph node involvement confirmed by FNA is almost certainly true involvement, i.e. there are almost no false positive diagnoses.

<figure id="fig-model-risks">

<figcaption>Figure 10: Histograms over the predicted risk of occult contralateral LNL II (left), III (middle), and IV (right) invovlement, shown for some combinations of T-category, tumor lateralization, and clinical LNL diagnoses. All LNLs not explicitly mentioned in the legend, including the LNL for which the risk of occult disease was computed, where assumed to be clinically negative, based on CT imaging (specificity 76%, sensitivity 81%).</figcaption>
</figure>

<!-- @Fig-model-risks shows this predicted risk of occult disease in the contralateral LNLs II, III, and IV for interesting combinations of these three risk factors. This risk is computed *given* a CT diagnosis assessing the per-LNL clinical involvement for which we assumed a sensitivity of 81% and a specificity of 76%. For the involvement in the LNls III and IV contralaterally, we also cosidered how the risk would be affected if the upstream LNL was confirmed to be metastatic using fine needle aspiration (FNA) with a specificity of 98% and a sensitivity of 80% [@de_bondt_detection_2007]. In the figure legends, this is noted e.g. for the orange historgam over the risk of contralateral LNL IV involvement as `c:III FNA+`, meaning that the contralateral LNL III was pathologically confirmed to harbor metastases via FNA.-->

## Contralateral LNL II

<a href="#fig-model-risks" class="quarto-xref">Figure 10</a> (left panel) shows the predicted risk of occult disease in contralateral LNL II. The most important variable impacting the prediction for contralateral level II involvement in our model is the tumor’s lateralization. A patient with a clearly lateralized early T-category tumor and a clinically N0 neck is predicted to have a 1-2% <!-- TODO: make this dynamic, and include this scenario in the plot --> risk for occult disease in contralateral LNL II. For a patient with a clearly lateralized early T-category tumor but *with* mid-sagittal extension, the risk increases to almost 7%. <!-- TODO: make this dynamic -->

Also advanced T-category increases the risk of occult disease but plays a lesser role: Considering the scenario of a tumor that crosses the midline and an ipsilateral neck where at least LNL III is clinically involved, the risk for occult contralateral LNL II disease is around 8.5%. <!-- TODO: make this dynamic --> For the same scenario and an advanced T-category tumor, the risk increases to 11%. <!-- TODO: make this dynamic -->

Lastly, the predicted risk is also correlated via the time-steps to the degree of ipsilateral involvement. Changing the aforementioned scenario (advanced T-category, midline extension, ipsilateral LNL III clinically involved) to one where the patient presents with a clinically N0 neck, the risk for occult disease in the contralateral LNL II falls from 11% <!-- TODO: make this dynamic --> to 9.5%. <!-- TODO: make this dynamic -->

Taken together, T-category and ipsilateral involvement may still considerably impact the risk prediction for contralateral involvement: In <a href="#fig-model-risks" class="quarto-xref">figure 10</a> the scenarios underlying the orange (6.5% <!-- TODO: make this dynamic -->) and the red (11% <!-- TODO: make this dynamic -->) histograms differ in T-category (early vs advanced) and clinical diagnosis (N0 vs ipsilateral at least LNL III involved).

## Contralalteral LNL III

As shown in the center subplot of <a href="#fig-model-risks" class="quarto-xref">figure 10</a>, the risk for occult disease in the contralateral LNL III may only cross a 5% threshold if the upstream level II is clinically involved, too. If the clinical diagnosis of the contralateral upstream level II is pathologically confirmed by FNA the risk increases by another percentage point<!-- TODO: make dynamic -->. When the tumor additionally crosses the mid-sagittal line, the risk may reach around 7%<!-- TODO: make dynamic -->.

## Contralateral LNL IV

Our model’s prediction for the contralateral LNL IV show that even in the most extreme case, with an advanced T-category tumor extending over the mid-sagittal line as well as metastases in every ipsilateral LNL and every contralateral upstream level, the risk stays just below 3%<!-- TODO: make dynamic -->. We only observe a risk of 5-6% when the involvement of the upstream level III is pathologically confirmed after taking a biopsy using FNA. This is because the model’s prior probability for contralateral LNL IV involvement is so low that even given a clinical diagnosis in that level, a false positive in LNL III is a more likely explanation. This possibility, however, drops virtually to zero when tumor cells are pathologically confirmed in level III.

# Discussion

## Summary

In this work we present a formalism to model the ipsi- and contralateral lymphatic involvement of oropharyngeal SCC patients. An ipsilateral model has been previously developed and published ([Ludwig et al. 2021](#ref-ludwig_hidden_2021); [Ludwig, Schubert, Barbatei, Bauwens, Hoffmann, et al. 2023](#ref-ludwig_modelling_2023)). Based on this, we introduce an extension that leaves the ipsilateral model untouched, but extends it to the contralateral side in an intuitive and comprehensible manner.

The model’s performance w.r.t. its ability to describe the data on contralateral nodal involvement was evaluated based on a dataset of 833<!-- TODO: make this dynamic --> patients from four institutions. To the best of our knowledge, this is the first time lymphatic tumor progression has been modelled in such detail and to such an extent. While there was work on regional tumor spread, these models were conceptually different, more limited in the LNLs for which they make predictions, and not trained with real patient data ([Benson, Whipple, and Kalet 2006](#ref-benson_markov_2006); [Jung et al. 2017](#ref-jung_development_2017)).

The model takes the clincal diagnosis of the lymph node levels, the primary tumor’s T-category, as well as lateralization into account when predicting the personalized risk for occult disease in any LNL of interest. Owing to the relatively few parameters, the model is highly interpretable and every parameter can be intuitively explained, while still offering good accuracy in its predictions.

## Implications for Contralateral Elective Nodal Treatment

The predictions of this model are being used to inform a clinical trial on volume deescalation at the University Hospital Zurich \[citation needed<!-- TODO: add -->\]. When accepting a 5% risk of occult disease in any given LNL, the model suggests the following contralateral elective irradiation (assuming the respective LNL appears clinically healthy):

-   LNL II when the tumor extends over the mid-sagittal line
-   LNL III when LNL II is clinically involved, may need confirmation via FNA
-   LNL IV only when upstream level III pathologically confirmed

# Acknowledgement

This work was supported by:

-   the Clinical Research Priority Program “Artificial Intelligence in Oncological Imaging” of the University of Zurich
-   the Swiss Cancer Research Foundation under grant number KFS 5645-08-2022

# Contralateral Prevalence of Involvement

In [6]:
lnl_cols = get_lnl_cols("contra", lnls=["I", "II", "III", "IV"])
num_ipsi_inv = raw[get_lnl_cols("ipsi")].sum(axis="columns")

contra_inv = raw[lnl_cols].copy()
contra_inv.columns = contra_inv.columns.droplevel([0,1])
contra_inv["t_stage"] = raw[COL.t_stage].apply(lambda x: "early" if x <= 2 else "advanced")
contra_inv["ipsi"] = num_ipsi_inv.map(lambda x: str(x) if x <= 1 else "≥ 2")
contra_inv["midext"] = raw[COL.midext]

grouped = contra_inv.groupby(by=["t_stage", "ipsi", "midext"])
num_involved = grouped.sum()
total = grouped.count()
percent_involved = num_involved / total
idx = total.index.rename(["T-cat.", "ipsi", "Mid. ext."])

total.index = idx
num_involved.index = idx
percent_involved.index = idx

involved = num_involved.join(
  100 * percent_involved,
  rsuffix=" (%)",
).sort_index(axis="columns")
involved.columns = pd.MultiIndex.from_product(
  [["I", "II", "III", "IV"],
   ["n", "%"]],
  names=["LNL", ""]
)
involved["total", "n"] = total["I"]
(
  involved
  .reset_index()
  .sort_values("ipsi", ascending=True)
  .sort_values("T-cat.", ascending=False)
  .style
  .format(precision=2)
  .apply(right_align, subset=involved.columns[0:])
  .apply(highlight_t_stage, subset=["T-cat."])
  .apply(highlight_bool, subset=["Mid. ext."])
  .apply(highlight, subset=["ipsi"], mapping={
    "0": COLORS["green"],
    "1": COLORS["orange"],
    "≥ 2": COLORS["red"],
  })
  .hide()
)

Benson, Noah, Mark Whipple, and Ira J Kalet. 2006. “A Markov Model Approach to Predicting Regional Tumor Spread in the Lymphatic System of the Head and Neck.” *AMIA ... Annual Symposium Proceedings. AMIA Symposium*, 31–35.

Biau, Julian, Michel Lapeyre, Idriss Troussier, Wilfried Budach, Jordi Giralt, Cai Grau, Joanna Kazmierska, et al. 2019. “Selection of Lymph Node Target Volumes for Definitive Head and Neck Radiation Therapy: A 2019 Update.” *Radiotherapy and Oncology* 134 (May): 1–9. <https://doi.org/10.1016/j.radonc.2019.01.018>.

Chao, K. S.Clifford, Franz J Wippold, Gokhan Ozyigit, Binh N Tran, and James F Dempsey. 2002. “Determination and Delineation of Nodal Target Volumes for Head-and-Neck Cancer Based on Patterns of Failure in Patients Receiving Definitive and Postoperative IMRT.” *International Journal of Radiation Oncology\*Biology\*Physics* 53 (5): 1174–84. <https://doi.org/10.1016/S0360-3016(02)02881-X>.

De Bondt, R, and Others. 2007. “Detection of Lymph Node Metastases in Head and Neck Cancer: A Meta-Analysis Comparing US, USgFNAC, CT and MR Imaging.” *Eur. J. Radiol.* 64: 266–72. <https://doi.org/10.1016/j.ejrad.2007.02.037>.

Eisbruch, Avraham, Robert L. Foote, Brian O’Sullivan, Jonathan J. Beitler, and Bhadrasain Vikram. 2002. “Intensity-Modulated Radiation Therapy for Head and Neck Cancer: Emphasis on the Selection and Delineation of the Targets.” *Seminars in Radiation Oncology* 12 (3): 238–49. <https://doi.org/10.1053/srao.2002.32435>.

Ferlito, Alfio, Carl E. Silver, and Alessandra Rinaldo. 2009. “Elective Management of the Neck in Oral Cavity Squamous Carcinoma: Current Concepts Supported by Prospective Studies.” *British Journal of Oral and Maxillofacial Surgery* 47 (1): 5–9. <https://doi.org/10.1016/j.bjoms.2008.06.001>.

Foreman-Mackey, Daniel, David W. Hogg, Dustin Lang, and Jonathan Goodman. 2013. “Emcee: The MCMC Hammer.” *\\Pasp* 125 (925): 306. <https://doi.org/10.1086/670067>.

Grégoire, Vincent, Kian Ang, Wilfried Budach, Cai Grau, Marc Hamoir, Johannes A. Langendijk, Anne Lee, et al. 2014. “Delineation of the Neck Node Levels for Head and Neck Tumors: A 2013 Update. DAHANCA, EORTC, HKNPCSG, NCIC CTG, NCRI, RTOG, TROG Consensus Guidelines.” *Radiotherapy and Oncology* 110 (1): 172–81. <https://doi.org/10.1016/j.radonc.2013.10.010>.

Grégoire, Vincent, Peter Levendag, Kian K. Ang, Jacques Bernier, Marijel Braaksma, Volker Budach, Cliff Chao, et al. 2003. “<span class="nocase">CT-based</span> Delineation of Lymph Node Levels and Related CTVs in the Node-Negative Neck: DAHANCA, EORTC, GORTEC, NCIC,RTOG Consensus Guidelines.” *Radiotherapy and Oncology* 69 (3): 227–36. <https://doi.org/10.1016/j.radonc.2003.09.011>.

Grégoire, V, and Others. 2018. “Delineation of the Primary Tumour Clinical Target Volumes (CTV-P) in Laryngeal, Hypopharyngeal, Oropharyngeal and Oral Cavity Squamous Cell Carcinoma: AIRO, CACA, DAHANCA, EORTC, GEORCC, GORTEC, HKNPCSG, HNCIG, IAG-KHT, LPRHHT, NCIC CTG, NCRI, NRG Oncology, PHNS, SBRT, SOMERA, SRO, SSHNO, TROG Consensus Guidelines.” *Radiother. Oncol.* 126: 3–24. <https://doi.org/10.1016/j.radonc.2017.10.016>.

Jung, Hyunggu, Anthony Law, Eli Grunblatt, Lucy L. Wang, Aaron Kusano, Jose L. V. Mejino, and Mark E. Whipple. 2017. “[Development of a Novel Markov Chain Model for the Prediction of Head and Neck Squamous Cell Carcinoma Dissemination](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5333213).” *AMIA Annual Symposium Proceedings* 2016 (February): 1832–39.

Kyzas, P A, and Others. 2008. “<span class="nocase">18F-fluorodeoxyglucose</span> Positron Emission Tomography to Evaluate Cervical Node Metastases in Patients with Head and Neck Squamous Cell Carcinoma: A Meta-Analysis.” *J. Natl Cancer Inst.* 100: 712–20. <https://doi.org/10.1093/jnci/djn125>.

Ludwig, Roman, Jean-Marc Hoffmann, Bertrand Pouymayou, Grégoire Morand, Martina Broglie Däppen, Matthias Guckenberger, Vincent Grégoire, Panagiotis Balermpas, and Jan Unkelbach. 2022. “A Dataset on Patient-Individual Lymph Node Involvement in Oropharyngeal Squamous Cell Carcinoma.” *Data in Brief* 43 (August): 108345. <https://doi.org/10.1016/j.dib.2022.108345>.

Ludwig, Roman, Bertrand Pouymayou, Panagiotis Balermpas, and Jan Unkelbach. 2021. “A Hidden Markov Model for Lymphatic Tumor Progression in the Head and Neck.” *Scientific Reports* 11 (1): 12261. <https://doi.org/10.1038/s41598-021-91544-1>.

Ludwig, Roman, Adrian Schubert, Dorothea Barbatei, Lauence Bauwens, Jean-Marc Hoffmann, Sandrine Werlen, Olgun Elicin, et al. 2023. “Modelling the Lymphatic Metastatic Progression Pathways of OPSCC from Multi-Institutional Datasets.” arXiv. <https://doi.org/10.48550/arXiv.2312.11270>.

Ludwig, Roman, Adrian Schubert, Dorothea Barbatei, Laurence Bauwens, Sandrine Werlen, Olgun Elicin, Matthias Dettmer, et al. 2023. “A Multi-Centric Dataset on Patient-Individual Pathological Lymph Node Involvement in Head and Neck Squamous Cell Carcinoma.” *Data in Brief*, December, 110020. <https://doi.org/10.1016/j.dib.2023.110020>.

Nelson, Benjamin, Eric B. Ford, and Matthew J. Payne. 2013. “RUN DMC: AN EFFICIENT, PARALLEL CODE FOR ANALYZING RADIAL VELOCITY OBSERVATIONS USING *N* -BODY INTEGRATIONS AND DIFFERENTIAL EVOLUTION MARKOV CHAIN MONTE CARLO.” *The Astrophysical Journal Supplement Series* 210 (1): 11. <https://doi.org/10.1088/0067-0049/210/1/11>.

Pouymayou, Bertrand, Panagiotis Balermpas, Oliver Riesterer, Matthias Guckenberger, and Jan Unkelbach. 2019. “A Bayesian Network Model of Lymphatic Tumor Progression for Personalized Elective CTV Definition in Head and Neck Cancers.” *Physics in Medicine & Biology* 64 (16): 165003. <https://doi.org/10.1088/1361-6560/ab2a18>.

ter Braak, Cajo J. F., and Jasper A. Vrugt. 2008. “Differential Evolution Markov Chain with Snooker Updater and Fewer Chains.” *Statistics and Computing* 18 (4): 435–46. <https://doi.org/10.1007/s11222-008-9104-9>.

Vorwerk, Hilke, and Clemens F. Hess. 2011. “Guidelines for Delineation of Lymphatic Clinical Target Volumes for High Conformal Radiotherapy: Head and Neck Region.” *Radiation Oncology* 6 (1): 97. <https://doi.org/10.1186/1748-717X-6-97>.